# Preprocesamiento datos Tiendanube

Esta notebook se encarga de preprocesar los datos con el fin de que puedan ser utilizados para el entrenamiento de un modelo predictivo (clasificador)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Lectura de datos

In [ ]:
df_data = pd.read_csv('test_data.csv', delimiter=';', index_col=0)
df_data.head()

In [ ]:
df_data.describe()

## Normalización de las variables numericas

El objetivo es aplicar una técnia de normalización sobre las variables numéricas para que todas ellas se muevan en el mismo rango [0;1]

Las variables a normalizar son:

* **total_events_on_iOS**: registro de la cantidad de eventos que tuvo el trial por un dispositivo con sistema operativo Android
* **total_events_on_Web**: registro de la cantidad de eventos que tuvo el trial por una computadora
* **total_events_on_Android**: registro de la cantidad de eventos que tuvo el trial por un dispositivo con sistema operativo iOS
* **admin_visits**: cantidad de visitas que registró el administrador 
* **intercom_conversations**: cantidad de conversaciones que registró la tienda con el equipo de soporte
* **products_with_description**: cantidad de productos que poseen descripción 
* **total_products_with_images**: cantidad de productos que poseen imagen
* **total_product_categories**: cantidad de categorías de productos creadas

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Creamos un objeto escalador min-max
min_max_scaler = MinMaxScaler()

In [ ]:
#Definimos las variables a normarlizar
columnas_a_normalizar = [ 
    'admin_visits',
    'intercom_conversations',
    'products_with_description',
    'total_products_with_images',
    'total_product_categories',
    'total_events_on_Android',
    'total_events_on_Web',
    'total_events_on_iOS',
]

In [ ]:
#Creamos un dataset auxiliar
df_data_numerica_norm = df_data[columnas_a_normalizar]
df_data_numerica_norm.head()

In [ ]:
#Escalamos los datos
df_data_numerica_norm = pd.DataFrame(min_max_scaler.fit_transform(df_data_numerica_norm), columns=df_data_numerica_norm.columns)
df_data_numerica_norm.head()

### One hot encoder para variables categoricas

Para las variables categóricas se aplica la técnica de *One Hot Encoder* que permite mapear a las variables categóricas en un vector booleano.

Las variables a mapear son:

* **country**: país de origen del trial
* **creation_platform**: plataforma por la cual se creó la tienda 
* **source_pulido**: origen de la tienda por campaña de marketing
* **creation_weekday**: día de la semana en que se creó la tienda

In [ ]:
columnas_categoricas =['country', 'creation_platform', 'source_pulido', 'creation_weekday']

In [ ]:
df_data_categorica = df_data[columnas_categoricas]
df_data_categorica['creation_weekday'] = df_data_categorica['creation_weekday'].apply(lambda x : str(x))
df_data_categorica.head()

In [ ]:
df_data_categorica =pd.get_dummies(df_data_categorica, prefix=columnas_categoricas)
df_data_categorica.head()

## Concatenar las variables

A continuación se unen las ambos tipos de variables en un solo dataframe

In [ ]:
df_procesado = pd.concat([df_data_numerica_norm, df_data_categorica], axis=1)
df_procesado['target'] = df_data['target'] 
df_procesado.head()

In [ ]:
df_procesado.to_csv('data_procesada.csv', index=False)

Luego se extrae una muestra representativa con el fin de poder entrenar y analizar rápidamente diversos tipos de **clasificadores**

In [ ]:
df_sample = df_procesado.sample(frac=0.2)
df_sample.shape

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x='target', data=df_sample)

In [ ]:
df_sample.to_csv('dataset_desbalanceado.csv', index=False)

### Dataset balanceado

Dado que se trata de una dataset desbalanceado hacia la **clase 0** (es decir, aquellos usuarios que no se pasarán a *payment*), a continuación se genera un dataset equilibrado descartando instancias de la **clase 0** (pero tratando de mantener las particularidades de dicha clase). 

Esta estrategia, si bien implica no aprovechar el total de datos disponibles, permite equilibrar el dataset en favor de la **clase 1** (es decir, aquellos usuarios que se pasarán a *payment*) que es nuestra clase de interés. De esta manera los clasificadores entrenados no se dejarán influenciar por la tendencia del dataset original

Primero, dividimos el dataframe en dos: uno para la **clase 0** y otro para la **clase 1**

In [ ]:
df_positivo = df_procesado[df_procesado['target'] == 1]
df_positivo.head()

In [ ]:
df_negativo = df_procesado[df_procesado['target'] == 0]
df_negativo.head()

Luego, se toma una muestra de dataframe de la **clase 0** que tenga un tamaño similar al dataframe de la **clase 1**

In [ ]:
clases_positivas = df_positivo.shape[0]
clases_positivas

In [ ]:
df_negativo_muestra = df_negativo.sample(n= (clases_positivas + int(clases_positivas*0.1)))
df_negativo_muestra.head()

Finalemente se unen en un solo dataset

In [ ]:
df_balanceado = pd.concat([df_positivo, df_negativo_muestra])
df_balanceado = df_balanceado.sort_index()
df_balanceado.head()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x='target', data=df_balanceado)

In [ ]:
df_balanceado.to_csv('dataset_balanceado.csv', index=False)